In [27]:
import pandas as pd
from scipy.stats import ranksums, pearsonr

In [28]:
# Load the data
Pla_gene = pd.read_csv("/home/yang/PM_AD/8_IL1RL1_FCER1G/Placenta_gene_exp.txt", index_col=0)
Clinical = pd.read_excel("/home/yang/PM_AD/1_Raw_data/placenta_CB_transcriptome_methylation_Clinical_231201.xlsx", sheet_name="임상정보_")

In [29]:
# Perform Wilcoxon rank-sum test between two groups 
def wilcox_ranksum_test(tmp1, Case, Control):
    df = tmp1.copy()

    # Subset data for Case  and Control
    group_G4 = df[df.index == Case]
    group_G3 = df[df.index == Control]

    p_values = []
    t_values = []

    # Perform Wilcoxon rank-sum test 
    for column in group_G4.columns:
        if column != 'placenta_metylation_transcriptome':  # Exclude specific column
            statistic, p_value = ranksums(group_G4[column], group_G3[column])
            p_values.append(p_value)
            t_values.append(statistic)

    # Create a DataFrame with p-values and t-values 
    result_df = pd.DataFrame({"P": p_values, "T": t_values}, index=df.columns)
    
    return result_df

# Perform Wilcoxon rank-sum test for the gene expression data (G4 PM-high_AD vs G3 PM-high healthy)
Pla_gene_res = wilcox_ranksum_test(Pla_gene.set_index("Group"), "G4", "G3")

In [30]:
# Pearson correlation function 
def Correlation_pearson(data, length):
    result_df_r = pd.DataFrame(index=data.iloc[:, length:].columns, columns=data.iloc[:, :length].columns)
    result_df_p = pd.DataFrame(index=data.iloc[:, length:].columns, columns=data.iloc[:, :length].columns)

    # Calculate Pearson correlation and p-values for each pair of columns
    for column1 in data.iloc[:, :length].columns:
        for column2 in data.iloc[:, length:].columns:
            x = data[column1]
            y = data[column2]
            valid_indices = ~x.isnull() & ~y.isnull()  # Filter out NaN values
            x_valid = x[valid_indices]
            y_valid = y[valid_indices]
            r, p_value = pearsonr(x_valid, y_valid)  # Compute Pearson correlation and p-value
            result_df_r.loc[column2, column1] = r
            result_df_p.loc[column2, column1] = p_value

    return result_df_p, result_df_r


# Filter the gene expression data to include FCER1G and all other columns
columns = ['FCER1G'] + [col for col in Pla_gene.columns if col != 'FCER1G']
df = Pla_gene.loc[:, columns]
df = df.drop(columns='Group')

# Calculate Pearson correlation for FCER1G gene
FCER1G_p, FCER1G_r = Correlation_pearson(df, length=1)

# Combine the p-values and correlation coefficients into one DataFrame
FCER1G_gene_cor = pd.concat([FCER1G_p.rename(columns={"FCER1G": "Pearson_P"}), 
                              FCER1G_r.rename(columns={"FCER1G": "Pearson_R"})], axis=1)


In [31]:
# Retrieve gene sets from Gene Ontology (GO), Reactome, and KEGG databases
GO_gene_set = pd.read_csv("/home/yang/PM_AD/Submission/Figure_3/Figure3A/GO_Gene_set.txt", index_col=3)
Reactome_gene_set = pd.read_csv("/home/yang/PM_AD/Submission/Figure_3/Figure3A/Reactome_DEG_Gene_set.txt", index_col=3)
KEGG_gene_set = pd.read_csv("/home/yang/PM_AD/Submission/Figure_3/Figure3A/KEGG_Gene_set.txt", index_col=3)

# Combine the gene sets
Gene_set = pd.concat([GO_gene_set, Reactome_gene_set, KEGG_gene_set], axis=0)

# Filter the gene set to include specific biological processes and pathways
Gene_set = Gene_set.loc[['innate immune response',
                         'adaptive immune response',
                         'immune effector process',
                         'positive regulation of immune system process',
                         'inflammatory response',
                         'myeloid leukocyte activation',
                         'myeloid leukocyte migration',
                         'leukocyte chemotaxis',
                         'leukocyte mediated immunity',        
                         'response to cytokine',
                         'cytokine production',
                         'cytokine-mediated signaling pathway',
                         'response to reactive oxygen species',
                         'NADPH oxidase complex',
                         'superoxide-generating NAD(P)H oxidase activity',
                         'transmembrane receptor protein kinase activity',
                         'positive regulation of protein tyrosine kinase activity',
                         'protein kinase C activity',
                         'protein kinase C binding',
                         'Homo sapiens: PI3K/AKT Signaling in Cancer',
                         'MAPK signaling pathway'],]

# Extract the list of genes from the Gene Set
genes_list = Gene_set['gene'].str.split(',').explode().str.strip()
unique_genes = genes_list.unique() # Remove duplicates
final_gene_list = list(unique_genes)
Pathway_gene_df = pd.DataFrame(index=final_gene_list)


In [32]:
# Merge FCER1G correlation results with the final gene list
FCER1G_cor_downstream = pd.merge(Pathway_gene_df, FCER1G_gene_cor, left_index=True, right_index=True, how='inner')
FCER1G_cor_downstream = pd.merge(Pla_gene_res, FCER1G_cor_downstream, left_index=True, right_index=True, how='inner')

# Apply filters: Pearson correlation (FCER1G) > 0 and Wilcoxon test statistic > 0
FCER1G_cor_downstream = FCER1G_cor_downstream[(FCER1G_cor_downstream['Pearson_R'] > 0) & (FCER1G_cor_downstream['T'] > 0)]
FCER1G_cor_downstream


,P,T,Pearson_P,Pearson_R
AKNA,0.133327,1.501111,0.102683,0.238431
NPR2,0.772830,0.288675,0.699957,0.057084
IGKV1-39,0.043308,2.020726,0.935203,0.012052
GBP3,0.839860,0.202073,0.154009,0.208984
NKAP,0.452920,0.750555,0.321749,0.146102
...,...,...,...,...
MIR105-1,0.032663,2.136196,0.619451,0.073523
IRAK1,0.525373,0.635085,0.534424,0.091905
FLNA,0.644167,0.461880,0.001513,0.445379
MPP1,0.644167,0.461880,0.003466,0.413734


In [36]:
# FCER1G_cor_downstream.to_csv("FCER1G_score_input.txt")

In [37]:
# We then calculate the score using the GSVA function, as detailed in the 2_Calculate_FCER1G_score.R